In [7]:
from udpipe2_tuples_generator import run_with_threads, get_all_text_from_folder, run_sequential, break_text_into_sentences

In [10]:
run_sequential("Eu gosto de cachorros.")

[]

In [2]:
# #  Read the g1 dataset.
# g1 = pd.read_csv('../data/csvs/g1_news_10000.csv')[['title', 'content']]

# # content will be the title + '.\n' + content
# g1['content'] = g1['title'] + '.\n' + g1['content']
# g1.drop(columns=['title'], inplace=True)

# run_with_threads(24, g1['content'].values, 'g1_tuples_udpipe')

In [2]:
# Read every file in the ../data/text_files/fake.br-corpus-true folder and create a dataframe with the texts.
import os
import re
import pandas as pd

fake_br_corpus_true_df = get_all_text_from_folder('../data/text_files/fake.br-corpus-true')
print(f'There are {fake_br_corpus_true_df.shape[0]} texts in the fake.br-corpus-true dataset.')
fake_br_corpus_true_df.drop_duplicates(inplace=True)
fake_br_corpus_true_df.dropna(inplace=True)
fake_br_corpus_true_df.reset_index(drop=True, inplace=True)
print(f'There are {fake_br_corpus_true_df.shape[0]} texts in the fake.br-corpus-true dataset after removing duplicates and nas.')
fake_br_corpus_true_df.to_csv('../data/csvs/fake_br_corpus_true.csv', index=False)

There are 8631 texts in the fake.br-corpus-true dataset.
There are 8625 texts in the fake.br-corpus-true dataset after removing duplicates and nas.


In [3]:
test_true_df = get_all_text_from_folder('../data/text_files/full_texts/true')
test_true_df.drop_duplicates(inplace=True)
test_true_df.dropna(inplace=True)
test_true_df.reset_index(drop=True, inplace=True)
print(f'There are {test_true_df.shape[0]} texts in the fake_br_original_true_df dataset after removing duplicates and nas.')

# check if there is any text in both datasets
number_of_repetitions = fake_br_corpus_true_df['content'].isin(test_true_df['content']).sum()
print(f'There are {number_of_repetitions} texts in both datasets.')

# Remove the texts that are in both datasets from the fake_br_corpus_true_df dataset.
fake_br_corpus_true_df = fake_br_corpus_true_df[~fake_br_corpus_true_df['content'].isin(test_true_df['content'])]
fake_br_corpus_true_df.reset_index(drop=True, inplace=True)
print(f'There are {fake_br_corpus_true_df.shape[0]} texts in the fake_br_corpus_true_df dataset after removing the texts that are in both datasets.')
fake_br_corpus_true_df.head()


There are 3599 texts in the fake_br_original_true_df dataset after removing duplicates and nas.
There are 90 texts in both datasets.
There are 8535 texts in the fake_br_corpus_true_df dataset after removing the texts that are in both datasets.


,content
0,o legado de che guevara 50 anos após sua morte...
1,polícia prende foragido da penitenciária de pe...
2,mp diz que novo bairro na região norte de pira...
3,brasilprev bloqueia cerca de r$ 9 milhões em p...
4,moro determina que r$ 9 milhões em planos de p...


In [5]:
number_of_sentences = 0
for text in fake_br_corpus_true_df['content'].values:
    number_of_sentences += len(break_text_into_sentences(text))

print(f'There are {number_of_sentences} sentences in the fake_br_corpus_true_df dataset.')

There are 237336 sentences in the fake_br_corpus_true_df dataset.


In [ ]:
run_with_threads(24, fake_br_corpus_true_df['content'].values, 'fake_br_corpus_tuples_udpipe')